In [1]:
%load_ext autoreload
%autoreload 2

from experiments.all import train_and_evaluate
from tsm.configuration import TSMConfigParser
import docker
import logging

logging.basicConfig(level=logging.INFO)

# load configuration
config = TSMConfigParser()
config.read("/home/pablo/tesis/tsm-pipeline/code/configs/pablo-bestia.cfg")
config.check()
docker_client = docker.from_env()

In [2]:
from scripts.docker import read_dbs_dataset
from numpy.random import RandomState

path_dbs = read_dbs_dataset("/home/pablo/tesis/tsm-pipeline/experiments/datasets/path_1.txt")
# Using a fixed random state to have reproducible results
rand = RandomState(seed=5)

print(len(path_dbs))

336


In [3]:
from random import sample
from sklearn.model_selection import train_test_split

sample_20 = sample(path_dbs, 20)
train, test = train_test_split(sample_20, train_size=.5, shuffle=True, random_state=rand)
print("train %d, test %d" %(len(train), len(test)))

train 10, test 10


corrida de sanity check con path

In [4]:
import tempfile
res_dir = tempfile.mkdtemp()
print("using %s for 20 run" % (res_dir))

train_and_evaluate(
    config=config,
    results_dir=res_dir,
    query_type="path",
    train=train,
    test=test,
    docker_client=docker_client,
)

INFO:experiments.all:train_and_evaluate with train size 10, test size 10
INFO:experiments.all:Running training
INFO:scripts.docker:creating /tmp/tmpitud061x dir. Omitting error if already existing
INFO:scripts.docker:creating /tmp/log dir. Omitting error if already existing


using /tmp/tmpitud061x for 20 run


INFO:scripts.docker:running at container 08e1c9e9f1aee691c61e7140067623b7fa7ed29c21a0991deccc3541a9a3e096. Use `docker logs 08e1c9e9f1aee691c61e7140067623b7fa7ed29c21a0991deccc3541a9a3e096 --tail 10 --follow` to follow progress
INFO:scripts.docker:running combine scores
INFO:misc.combinescores:globbing folder: /tmp/tmpitud061x/*/reprScores.txt
INFO:misc.combinescores:working on 8 reprScores files
INFO:experiments.all:Running worse evaluation
INFO:scripts.evaluate:creating /tmp/tmpitud061x/worse dir. Omitting error if already existing
INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating tianxiangbing - mobile-select-area
INFO:scripts.evaluate:Evaluating alphagov - webchat-prototype
INFO:scripts.evaluate:Evaluating ckeditor - ckeditor5-dev
INFO:scripts.evaluate:Evaluating Klemek - snex.io
INFO:scripts.evaluate:Evaluating cdaniel - wardleymapstool
INFO:scripts.evaluate:Evaluating apache - couchdb-fauxton
INFO:scripts.evaluate:Evaluating k88hudson - w

,precision,recall,accuracy,alerts to recover (atr),alerts recovered,suprious alerts,projects with atr,avg atr per proj
0,0.0,0.0,0.0,1,0,3,1,1.0
